<a href="https://colab.research.google.com/github/INFINITY-RUBER/Curso_platzi_de_Deep_Learning_con_Pytorch/blob/main/Notebooks%20Clases/24.ImplementandoDecodingparaRedGeneradora.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
%cd '/gdrive/My Drive'

/gdrive/My Drive


In [4]:
!git clone https://github.com/aitorzip/PyTorch-CycleGAN.git

Cloning into 'PyTorch-CycleGAN'...
remote: Enumerating objects: 60, done.
remote: Total 60 (delta 0), reused 0 (delta 0), pack-reused 60
Unpacking objects: 100% (60/60), done.


In [3]:
%cd '/gdrive/My Drive/PyTorch-CycleGAN'

/gdrive/My Drive/PyTorch-CycleGAN


In [4]:
!ls
!pwd


datasets     download_dataset  models.py  README.md  train
datasets.py  LICENSE	       output	  test	     utils.py
/gdrive/My Drive/PyTorch-CycleGAN


In [ ]:
# BUIL DATASET  (construir el dataset) 
# summer2winter_yosemite : es el dataset que quiero bajar
%%sh
sh ./download_dataset summer2winter_yosemite

In [13]:
# MOVER EL DATASET A DRIVE
!mv datasets/summer2winter_yosemite /gdrive/My\ Drive/dl-pytorch/datasets/

In [5]:
!ls /gdrive/My\ Drive/dl-pytorch/datasets/

64x64_SIGNS	 cifar-10-batches-py	 summer2winter_yosemite
64x64_SIGNS.zip  cifar-10-python.tar.gz


In [ ]:
# SEGMENTOS DE CAPAS DEGENERATIVOS
import torch.nn as nn
import torch.nn.functional as F

class ResidualBlock(nn.Module):
  def __init__(self, in_features):
    super(ResidualBlock, self).__init__()
    
    conv_block = [ nn.ReflectionPad2d(1), #mejor padding (alarga y rellena )
                   nn.Conv2d(in_features, in_features, 3),
                   nn.InstanceNorm2d(in_features), # basnun BN para GANS
                   nn.ReLU(True),
                   nn.ReflectionPad2d(1), #mejor para consrvar distribucion de la imagen
                   nn.Conv2d(in_features, in_features, 3),
                   nn.InstanceNorm2d(in_features)
                 ]
    
    self.conv_block = nn.Sequential(*conv_block) # para apilar la lista
    def forward(self, x):
      return self.conv_block(x) + x #una idea poderosa


In [ ]:
# CLASE GENERADORA
class Generator(nn.Module):
  def __init__(self, input_nc, output_nc, n_residual_blocks=9):
    super(Generator,self).__init__()
    
    # Bloqueconvolucional
    model = [ nn.ReflectionPad2d(3),
            nn.Conv2d(input_nc, 64, F), # I - 7 + 6 /1 +1 = I
            nn.InstanceNorm2d(64),
             nn.ReLU(True)
            ]
    
    in_features = 64
    out_features = in_features * 2
    
    #Encoding
    for _ in range(2):
      model += [ nn.Conv2d(in_features, out_features, 3, stride=2, padding=1), # divide por 2 el tamaño de la imgangen de entrada.  I/2
                 nn.InstanceNorm2d(out_features),
                 nn.ReLU(True)
               ]
      in_features = out_features
      out_features = in_features*2
    
    #transformaciones residuales
    
    for _ in range(n_residual_blocks):
      model += [ResidualBlock(in_features)]
    
    #decoding
    
    out_features = in_features/2
    for _ in range(2):
      model += [ nn.ConvTransose2d(in_features, out_features, 3, stride=2, padding=1, output_padding=1), #2I
                 nn.InstanceNorm2d(out_features),
                 nn.ReLU(True)
               ]
      in_features = out_features
      out_feature = in_features //2
      
      #salida
      model += [ nn.ReflectionPad2d(3),
                 nn.Conv2d(64, output_nc, 7), #I
                 nn.Tanh()
               ]
      
      self.model = nn.Sequential(*model)
      
    def forward(self,x):
      return self.model(x)